In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/xgb-click-train/XGB_fold3_click.xgb
/kaggle/input/xgb-click-train/XGB_fold1_click.xgb
/kaggle/input/xgb-click-train/__results__.html
/kaggle/input/xgb-click-train/XGB_fold0_click.xgb
/kaggle/input/xgb-click-train/__notebook__.ipynb
/kaggle/input/xgb-click-train/__output__.json
/kaggle/input/xgb-click-train/XGB_fold2_click.xgb
/kaggle/input/xgb-click-train/XGB_fold4_click.xgb
/kaggle/input/xgb-click-train/custom.css
/kaggle/input/otto-train-and-test-data-for-local-validation/test_labels.parquet
/kaggle/input/otto-train-and-test-data-for-local-validation/id2type.pkl
/kaggle/input/otto-train-and-test-data-for-local-validation/train.parquet
/kaggle/input/otto-train-and-test-data-for-local-validation/type2id.pkl
/kaggle/input/otto-train-and-test-data-for-local-validation/test.parquet
/kaggle/input/val-click-candidates-features/click_candidate_with_features_p2.pqt
/kaggle/input/val-click-candidates-features/click_candidate_with_features_p3.pqt
/kaggle/input/val-click-candidates

In [2]:
test_candidates = pd.read_parquet('/kaggle/input/test-click-candidates-features/test_click_candidates.pqt')

In [3]:
test_candidates

,session,aids,item_item_count,item_user_count,item_buy_ratio,user_user_count,user_item_count,user_buy_ratio,feature_click,feature_cart,feature_buy
0,12899779,59625,11,10,0.000000,1,1,0.0,1,0,0
1,12899779,1253524,212,168,0.014151,1,1,0.0,0,0,0
2,12899779,737445,713,458,0.015428,1,1,0.0,0,0,0
3,12899779,438191,2908,1914,0.028886,1,1,0.0,0,0,0
4,12899779,731692,96,49,0.197917,1,1,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
16877728,14571581,831165,27032,15162,0.063036,1,1,0.0,0,0,0
16877729,14571581,1116095,38598,21952,0.109876,1,1,0.0,0,0,0
16877730,14571581,832192,79784,38196,0.130014,1,1,0.0,0,0,0
16877731,14571581,1469891,19492,10548,0.079674,1,1,0.0,0,0,0


In [4]:
import xgboost as xgb

for i in range(0, 167180300, 10000000):
    print(i)
    if i== 160000000:
        candidates = test_candidates[i:]
    else:
        candidates = test_candidates[i:i+10000000]
    preds = np.zeros(len(candidates))
    for fold in range(5):
        model = xgb.Booster()
        model.load_model(f'/kaggle/input/xgb-click-train/XGB_fold{fold}_click.xgb')
        #model.set_param({'predictor': 'gpu_predictor'})
        dtest = xgb.DMatrix(data=candidates.iloc[:, 2:])
        preds += model.predict(dtest)/5
    predictions = candidates[['session','aids']].copy()
    predictions['pred'] = preds

    predictions = predictions.sort_values(['session','pred'], ascending=[True,False]).reset_index(drop=True)
    predictions['n'] = predictions.groupby('session').aids.cumcount().astype('int8')
    predictions = predictions.loc[predictions.n<20]
    sub = predictions.groupby('session').aids.apply(list)
    sub = sub.to_frame().reset_index()
    sub.aids = sub.aids.apply(lambda x: " ".join(map(str,x)))
    sub.columns = ['session_type','labels']
    sub.session_type = sub.session_type.astype('str')+ '_clicks'
    
    if i==0:
        submission = sub
    else:
        submission = pd.concat([submission, sub], ignore_index=True)

0
10000000
20000000
30000000
40000000
50000000
60000000
70000000
80000000
90000000
100000000
110000000
120000000
130000000
140000000
150000000
160000000


In [5]:
submission

,session_type,labels
0,12899779_clicks,59625 3295 1660529 45290 225209 689970 1790770...
1,12899780_clicks,736515 582732 1142000 973453 1360606 579690 99...
2,12899781_clicks,199008 141736 918667 194067 57315 1066725 1792...
3,12899782_clicks,1494780 595994 834354 975116 476063 740494 889...
4,12899783_clicks,255297 1729553 1817895 198385 1114789 1754419 ...
...,...,...
1671798,14571577_clicks,1141710 970741 768043 86916 367734 557072 1187...
1671799,14571578_clicks,519105 1264437 815460 1139638 977826 735459 82...
1671800,14571579_clicks,739876 1445637 1434015 707204 1633220 557072 1...
1671801,14571580_clicks,202353 679257 888228 1627186 1117970 1682397 1...


In [6]:
submission.to_parquet('click_submission.pqt')